In [2]:
import pandas as pd
import numpy as np
import re
import math
import tabula
import camelot
import pdfminer
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
try:
    from cStringIO import StringIO
except ImportError:
    from io import StringIO 
import matplotlib.pyplot as plt 

In [33]:
#19
#12,14,17,22
f = r'C:\Users\rites\Desktop\mudracircle\bks_raw\Parsing_testing\Bank of Baroda\files2\bob19.pdf'
tables=tabula.read_pdf(f,
                              stream=True,
                              pages="all",
                              silent=True,
                              multiple_tables=True,
                              pandas_options={'header':None})

df = pd.DataFrame()
df = pd.concat([c for c in tables]).drop_duplicates()
# df.to_csv("check.csv",index = False)

try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index][0]
    df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('transaction').any(),axis=1)==True].index][0]
        df.columns=df.iloc[idx]; df=df.iloc[idx+1:,:]; df.reset_index(drop=True, inplace=True)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ][0]
            df.columns=df.iloc[idx] ; df=df.iloc[idx+1:,:] ; df.reset_index(drop=True,inplace=True)
        except:
            print("\nBOB Column Headers Missing"); pass
        
df=df[~df.index.isin(df[df.apply(lambda row: row.astype(str).str.lower().str.contains('page:|account status|total|reason for return|inward clg|opening balance|statement of a/c|statement summary|generated on|generated by|computer generated statement|not').any(), axis=1) ==True].index)]

if df.shape[1] > 1:
    if df.shape[1] == 5:
        df.columns = [0,1,2,3,4]
        df["Mix_data"] = df[0].astype(str)+df[1].astype(str)+df[2].astype(str)+df[3].astype(str)+df[4].astype(str)
        df = df[["Mix_data"]]

df.columns = ["Mix_data"]

for i,j in enumerate(df["Mix_data"]):
    if j.count("|") == 2:
        if df["Mix_data"][i+2].count("|") == 4:
            df["Mix_data"][i+1] = df["Mix_data"][i+1]+df["Mix_data"][i+2]
            df = df.drop(i+2)

split_data = df['Mix_data'].str.split("|")
data = split_data.to_list()
new_df = pd.DataFrame(data)
df = new_df.replace(r'^\s*$', np.nan, regex=True)

df = df[[1,2,3,4,5]]
df.columns = ["DATE", "DETAILS", "DEBIT", "CREDIT", "BALANCE"]

try:
    idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('date').any(),axis=1)==True].index]
    for i in idx:
        df = df.drop(i)
except:
    try:
        idx=[c for c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('balance').any(),axis=1)==True].index if c in df[df.apply(lambda row:row.astype(str).str.lower().str.contains('details').any(),axis=1)==True].index]
        for i in idx:
            df = df.drop(i)
    except:
        try:
            idx=[ c for c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('date').any(), axis=1) ==True].index if c in df[df.apply(lambda row: row.astype(str).str.lower().str.contains('remarks').any(), axis=1) ==True].index ]
            for i in idx:
                df = df.drop(i)
        except:
            print("\nExtra BOB Column Headers Missing"); pass

df.dropna(how = 'all', inplace = True)

try:
    bal=[c for c in df.columns if "BALANCE" in str(c).upper() ][0]
except: print("\nBalance coprint(lumns missing")   

df['flag']=df.iloc[:,-1].shift(1)
df=df[~df.index.isin(df[df.iloc[:,-1]==df.iloc[:,-2]].index)]

try:
    dat=[c for c in df.columns if "TRANSACTION DATE" in str(c).upper() ][0]
except:
    try:
        dat=[c for c in df.columns if "TXN DATE" in str(c).upper() ][0]
    except:
        try:
            dat=[c for c in df.columns if "DATE" in str(c).upper() ][0]
        except:
            print("\nDate column missing")   
            pass

try:
    chq=[c for c in df.columns if "CHQ" in str(c).upper() ][0]
except:
    try:
        chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
    except:
        df.insert(loc=0, column='CHEQUE', value=np.nan)
        #df["CHEQUE"] = np.nan
        chq=[c for c in df.columns if "CHEQUE" in str(c).upper() ][0]
        pass

# print(df.columns)

try:
    narr=[c for c in df.columns if "REMARKS" in str(c).upper() ][0]
except:
    try:
        narr=[c for c in df.columns if "PARTICULARS" in str(c).upper() ][0]
    except:
        try:
            narr=[c for c in df.columns if "DESCRIPTION" in str(c).upper() ][0]
        except:
            try:
                narr=[c for c in df.columns if "DETAILS" in str(c).upper() ][0]
            except:
                try:
                    narr=[c for c in df.columns if "NARRATION" in str(c).upper() ][0]
                except:
                    print("\nNarration column missing")
                    pass

try:
    wdl=[c for c in df.columns if "WITHDRAW" in str(c).upper() ][0]
except:
    try:
        wdl=[c for c in df.columns if "AMOUNT" in str(c).upper() ][0]
    except:
        try:
            wdl=[c for c in df.columns if "DEBIT" in str(c).upper() ][0]
        except:
            print("\nDebit column missing")
            pass

try:
    dep=[c for c in df.columns if "DEPOSIT" in str(c).upper() ][0]
except:
    try:
         dep=[c for c in df.columns if "CREDIT" in str(c).upper() ][0]
    except:
        print("\nCredit column missing")
        pass
    
df[dep]=df[dep].apply( lambda x: x.split(' ')[0] if type(x) == str else x )
df[wdl]=df[wdl].apply( lambda x: x.split(' ')[0] if type(x) == str else x )  
df[wdl]=df[wdl].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float) *-1
df[dep]=df[dep].astype(str).apply(lambda x: str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
df[bal]=df[bal].astype(str).apply(lambda x:str(x).replace("\r","").replace(",","").replace("Cr","").replace("Dr","")).astype(float)
df['Wdl1']=df[wdl].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","").replace("(Cr)","").replace("(Dr)","")).astype(float) *-1
df['Wdl1']=df['Wdl1'].fillna(df[dep].astype(str).apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","").replace("(Cr)","").replace("(Dr)","")).astype(float) )                

# df=df.T.drop_duplicates().T           
df[bal]=df[bal].apply(lambda x: str(x).replace(",","").replace("Cr","").replace("Dr","").replace("(Cr)","").replace("(Dr)","")).astype(float)                                 
df['flag']=df.iloc[:,0].astype(str)+df['Wdl1'].astype(str) +df[bal].astype(str)
df['flag2']=np.arange(len(df))

df.loc[ df[['flag']].duplicated(keep=False) , 'flag' ] = df['flag'] + df['flag2'].astype(str)  
df['flag']=df['flag'].apply( lambda row : np.nan if 'nannannan' in row else row ).fillna(method='ffill')        

df[narr]=df.groupby('flag')[narr].transform( lambda x :' '.join(x))  
df[narr] = df[narr].str.replace("nannannannan","")
# print(df.columns)
df=df.drop_duplicates(['flag'],keep='first').iloc[0:,:-3].reset_index(drop=True)
# print(df.columns)

df = df.replace(r'^\s*$', np.nan, regex=True)
print(dat)
df = df[[dat,chq,narr,wdl,dep,bal]]
df.columns = ["Xns Date","Cheque No","Narration","Debits","Credits","Balance"]

df.to_csv("check2.csv", index=False)

print("parsed")

DATE
parsed
